# Supp Exercise 3 - Range proofs with ZKPs

While the Exercise 3 shows a simple way of creating ZKPs using the docknetwork/crypto library, this notebook shows a more detailed version of using ZKPs.
It can help giving an intuition of how ZKPs work and what is possible with them.

You can find a detailed explanation here: [docknetwork/crypto Proof System](https://github.com/docknetwork/crypto/tree/main/proof_system).

---

## 1. Issuer - Setup and signing of the credential


In [ ]:
const JOHN_SNOW_BANK_IDENTITY = {
  fname: 'John',
  lname: 'Snow',
  highly_secretive: {
    secret: 'super-duper-secret',
    email: 'john.snow@example.com',
    SSN: '123-456789-0',
    'user-id': 'user:USR-22201'
  },
  location: {
    country: 'CH',
    city: 'Lausanne',
    address: 'C4DT, station 14'
  },
  timeOfBirth: 1662010849619,
  physical: {
    height: 170,
    weight: 78,
    race: 'dark elf'
  },
  score: 200
};

const SCHEMA = {
  fname: null,
  lname: null,
  highly_secretive: {
    secret: null,
    email: null,
    SSN: null,
    'user-id': null
  },
  location: {
    country: null,
    city: null,
    address: null
  },
  timeOfBirth: null,
  physical: {
    height: null,
    weight: null,
    race: null
  },
  score: null
};

In [ ]:
// Setting up the keys for the issuer

import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();
import { BBSPlusKeypairG2, BBSPlusSignatureParamsG1, Encoder, EncodeFunc, BBSPlusSignatureG1,
       getAdaptedSignatureParamsForMessages, BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

// Generating a keypair once signature parameters are created.

// Deterministically generated params
const label = stringToBytes("Awesome key pair");
const cryptoParams = BBSPlusSignatureParamsG1.generate(14, label);

const keypair = BBSPlusKeypairG2.generate(cryptoParams);
const sk = keypair.secretKey;
const pk = keypair.publicKey;

// Creating an encoder for the ZKPs

// The encoder has to be known and agreed upon by all system participants, i.e. signer, prover and verifier.
const encoders = new Map<string, EncodeFunc>();
encoders.set('score', Encoder.positiveIntegerEncoder());
const GlobalEncoder = new Encoder(encoders, Encoder.defaultEncodeFunc());

// Signing the credential

// The signing function will encode bytes to a field element as true is passed
const sig = BBSPlusSignatureG1.signMessageObject(JOHN_SNOW_BANK_IDENTITY, sk, label, GlobalEncoder);

// As the messages are not encoded, pass true to the verification function to make it encode messages before verifying the signature.
sig.signature.verifyMessageObject(JOHN_SNOW_BANK_IDENTITY, pk, label, GlobalEncoder);

const sigParams = getAdaptedSignatureParamsForMessages(cryptoParams, SCHEMA);

// Creating the setup which will be sent to the prover.

const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

const proverSetupParams: SetupParam[] = [];
proverSetupParams.push(SetupParam.legosnarkProvingKeyUncompressed(snarkProvingKey));

---

## 2. Holder - Zero Knowledge Proof

Here's we will provide a proof of the following: 
1. Selective disclosure of: User's firstName, lastName, location
2. Range proof: Score > 5

In [ ]:
// Both prover and verifier can independently create this struct
import { getRevealedAndUnrevealed, Statement, Statements, MetaStatements, ProofSpec, Witness, Witnesses, CompositeProof,
        WitnessEqualityMetaStatement, getIndicesForMsgNames, MetaStatement, QuasiProofSpec } from '@docknetwork/crypto-wasm-ts';

const revealedNames = new Set(['fname', 'physical.height', 'location.city'])

// Prover prepares messages it wishes to reveal and hide.
const [revealedMsgs, unrevealedMsgs, revealedMsgsDecoded] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedNames,
    GlobalEncoder
);

console.log("The following fields will be revealed:", revealedMsgsDecoded)

// Prover prepares messages it wishes to reveal and hide.
const revealedParametersB = new Set(['fname', 'lname', 'location.city'])

const [revealedMsgsB, unrevealedMsgsB, revealedMsgsDecodedB] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedParametersB,
    GlobalEncoder
);

// Create a BBS signature
const statementB1 = Statement.bbsPlusSignatureProverConstantTime(sigParams, revealedMsgsB, false);
const statementB2 = Statement.boundCheckLegoProverFromSetupParamRefs(1, 1000, 0);
const statementsB = new Statements([statementB1, statementB2]);

const scoreFieldIndex = getIndicesForMsgNames(['score'], SCHEMA)[0]  // 12
const witnessEq = new WitnessEqualityMetaStatement();
witnessEq.addWitnessRef(0, scoreFieldIndex);  // addWitnessRef(statementId, dataPointIDInWitness)
witnessEq.addWitnessRef(1, 0);

const metaStatementsB = new MetaStatements();
metaStatementsB.addWitnessEquality(witnessEq);

const proofSpecB = new ProofSpec(statementsB, metaStatementsB, proverSetupParams);
console.log("The proof specification is valid:", proofSpecB.isValid());

// Creating the witnesses

const witnessB1 = Witness.bbsPlusSignatureConstantTime(sig.signature, unrevealedMsgsB, false);
const witnessB2 = Witness.boundCheckLegoGroth16(sig.encodedMessages['score']);
const witnessesB = new Witnesses([witnessB1, witnessB2]);

const proof = CompositeProof.generate(proofSpecB, witnessesB);
console.log("This will be sent to the verifier:", revealedMsgsDecodedB, proof);

---

## 3. Verifier

In [ ]:
import {encodeRevealedMsgs} from '@docknetwork/crypto-wasm-ts'; 

// Verifier 
const verifierSetupParams: SetupParam[] = [];
verifierSetupParams.push(SetupParam.legosnarkVerifyingKeyUncompressed(snarkVerifyingKey));

const revealedMsgs1FromVerifier = encodeRevealedMsgs(revealedMsgsDecodedB, SCHEMA, GlobalEncoder);
// checkMapsEqual(revealedMsgs, revealedMsgs1FromVerifier);

const statement1 = Statement.bbsPlusSignatureVerifierConstantTime(sigParams, pk, revealedMsgs1FromVerifier, false)
const statement2 = Statement.boundCheckLegoVerifierFromSetupParamRefs(1, 1000, 0);

const statementsVerifier = new Statements([statement1, statement2]);

const verifierWitnessEq = new WitnessEqualityMetaStatement();
verifierWitnessEq.addWitnessRef(0, getIndicesForMsgNames(['score'], SCHEMA)[0]);
verifierWitnessEq.addWitnessRef(1, 0);

const metaStatements = new MetaStatements();
metaStatements.addWitnessEquality(verifierWitnessEq);

const proofSpecVerifier = new ProofSpec(statementsVerifier, metaStatements, verifierSetupParams);
proofSpecVerifier.isValid();

console.log("Got the following revealed fields:", revealedMsgsDecodedB);
console.log("Proof for financial score between 1 and 1000:", proof.verify(proofSpecVerifier).verified);